In [3]:
import micropip

We need to patch [pyfilesystem2](https://docs.pyfilesystem.org/en/latest/index.html) because pyodide currently doesn't implement all of the standard library. See [#38](https://github.com/brightway-lca/brightway-live/issues/38).

In [4]:
await micropip.install(
    'https://files.brightway.dev/fs-2.5.1-py2.py3-none-any.whl'
)

Manually set the directory where data is stored to avoid errors related to the browser virtual filesystem.

In [5]:
import os
os.environ["BRIGHTWAY_DIR"] = "/tmp/"

In [6]:
await micropip.install('bw2data==4.0.dev29', keep_going = True)
await micropip.install('bw2io==0.9.dev23', keep_going = True)
await micropip.install('bw2calc==2.0.dev14', keep_going = True)

In [7]:
import bw2data
import bw2calc
import bw2io

We can't import larger project backups yet - see [#37](https://github.com/brightway-lca/brightway-live/issues/37). But we can create a small sample database.

In [8]:
bw2io.add_example_database(searchable=False)

/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Extracted 4 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: assign_only_product_as_production
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 11 strategies in 0.00 seconds
Applying strategy: link_iterable_by_fields
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


/lib/python3.11/site-packages/bw2data/backends/base.py:50: TqdmMonitorWarning: tqdm:disabling monitor support (monitor_interval = 0) due to:
can't start new thread
  return tqdm(iterable)
100%|██████████| 9/9 [00:00<?, ?it/s]


Vacuuming database 
Created database: Mobility example


We can now do normal Brightway operations:

In [9]:
bw2data.databases

Databases dictionary with 1 object(s):
	Mobility example

In [10]:
for obj in bw2data.Database("Mobility example"):
    print(obj)

'Driving an combustion car' (kilometer, Europe, None)
'CO2' (kilogram, None, None)
'Electric car battery' (unit, GLO, ('Electric car battery',))
'Combustion car' (unit, GLO, ('Combustion car',))
'Electric car' (unit, GLO, ('Electric car',))
'Steel' (kilogram, GLO, None)
'Driving an electric car' (kilometer, Europe, None)
'Electricity' (kilowatt hour, GLO, None)
'Lithium' (kilogram, GLO, None)


In [11]:
co2 = bw2data.get_node(name="CO2")

In [12]:
gwp = bw2data.Method(("GWP", "simple"))
gwp.write([(co2.key, 1)])

In [ ]:
lca = bw2calc.LCA({bw2data.get_node(name="Electric car"): 